In [33]:
from sklearn.feature_extraction.text import TfidfVectorizer

import torch
from transformers import BertTokenizer, BertModel
import pickle

import spacy
import pandas as pd

In [34]:
# 2. Carreguem dades
DF = pd.read_pickle("data/datos_preprocesados.pkl")

Creem un dataframe amb el nom i la descripció

In [35]:
df = DF[["Name","Description"]]
df = df.drop_duplicates(subset='Name', keep='first').reset_index(drop=True)
DF = DF.drop(columns=["Description"])
#print(df)

TF-idf

In [36]:
# Crear el vectoritzador
vectorizer = TfidfVectorizer()

# Aplicar TF-IDF sobre la columna 'Description'
tfidf_matrix = vectorizer.fit_transform(df['Description'])

# Convertir la matriu en un DataFrame per veure-ho clar
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=vectorizer.get_feature_names_out())

# Afegir l'ID o index per context
tfidf_df['Name'] = df['Name']

In [37]:
#tfidf_df

In [38]:
DF_tfidf = DF.merge(tfidf_df, on="Name", how="left")

Word2Vec

In [39]:
# Carrega el model amb embeddings
nlp = spacy.load("en_core_web_md")

# Exemple: df = pd.DataFrame({'Name': [...], 'Description': [...]})

# Funció per calcular vector mitjà de la frase
def sentence_vector(text):
    doc = nlp(text)
    return doc.vector

# Aplicar a cada descripció
sentence_vecs = df['Description'].astype(str).apply(sentence_vector)
sentence_vecs_df = pd.DataFrame(sentence_vecs.tolist())
sentence_vecs_df.insert(0, 'Name', df['Name'])

In [40]:
#sentence_vecs_df

In [41]:
DF_W2V = DF.merge(sentence_vecs_df, on="Name", how="left")

Transformers Bert

In [42]:
# Suposem que tens el DataFrame df amb 'Description' i 'Name'
descriptions = df['Description'].astype(str).tolist()

# Carregar model i tokenizer BERT (base uncased, pots canviar a model en català si vols)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

# Funció per obtenir l'embedding d'una frase (mitjana dels tokens)
def get_embedding(text):
    inputs = tokenizer(text, return_tensors='pt', truncation=True, max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
    # outputs.last_hidden_state té la representació per cada token
    # Fem la mitjana sobre els tokens per tenir un vector fix (1 x 768)
    embeddings = outputs.last_hidden_state.mean(dim=1).squeeze()
    return embeddings.numpy()

# Generar embeddings per tots els textos (pot ser lent si és gran)
embeddings = [get_embedding(text) for text in descriptions]

# Convertir a DataFrame, 768 columnes per BERT base
embeddings_df = pd.DataFrame(embeddings)

# Afegir la columna Name
embeddings_df.insert(0, 'Name', df['Name'].values)

In [43]:
#embeddings_df

In [44]:
DF_TBert = DF.merge(embeddings_df, on="Name", how="left")


In [45]:
DF_tfidf.columns = DF_tfidf.columns.astype(str)

In [46]:
DF_tfidf.columns = DF_tfidf.columns.astype(str)

In [47]:
DF_TBert.columns = DF_TBert.columns.astype(str)


In [48]:
DF_tfidf.to_pickle(path="data/dades_TF-idf.pkl")
DF_W2V.to_pickle(path="data/dades_W2V.pkl")
DF_TBert.to_pickle(path="data/dades_TBert.pkl")

ACM

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
import prince

# Suposem que tens el DataFrame df amb 'Description' i 'Name'

# 1. Vectoritzar les frases en forma binària (presència/absència)
vectorizer = CountVectorizer(binary=True)
X = vectorizer.fit_transform(df['Description'].astype(str))

# Convertir a DataFrame per comoditat
X_df = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names_out())

# 2. Aplicar ACM amb prince
mca = prince.MCA(n_components=10, n_iter=3, copy=True, check_input=True, engine='sklearn', random_state=42)
mca = mca.fit(X_df)

# 3. Coordenades de les paraules (columnes)
word_coords = mca.column_coordinates(X_df)

# 4. Funció per obtenir el vector de frase com mitjana vectors paraules
def sentence_vector(sentence):
    words = sentence.lower().split()
    valid_words = [w for w in words if w in word_coords.index]
    if not valid_words:
        return [0]*word_coords.shape[1]
    return word_coords.loc[valid_words].mean(axis=0).values

# 5. Aplicar a totes les frases
sentence_vectors = df['Description'].astype(str).apply(sentence_vector).tolist()

# 6. Crear DataFrame amb resultats
sentence_vectors_df = pd.DataFrame(sentence_vectors)
sentence_vectors_df.insert(0, 'Name', df['Name'].values)

In [ ]:
sentence_vectors_df